In [1]:
import os

In [2]:
%pwd

'd:\\aiadventures\\Projects\\ml_complete_proj\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\aiadventures\\Projects\\ml_complete_proj'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
# os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/PranavUikey/ml_complete_proj.mlflow'
# os.environ['MLFLOW_TRACKING_USERNAME'] = 'PranavUikey'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'b164a979bd8248b638f9af447e9b059ea588b82a'

In [7]:
from ml_complete_proj.constants import *
from ml_complete_proj.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config1 = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        config2 = self.config.mlflow_info

        os.environ['MLFLOW_TRACKING_URI'] = config2.tracking_uri
        os.environ['MLFLOW_TRACKING_USERNAME'] = config2.tracking_username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = config2.tracking_password

        create_directories([config1.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config1.root_dir,
            test_data_path=config1.test_data_path,
            model_path = config1.model_path,
            all_params=params,
            metric_file_name = config1.metric_file_name,
            target_column = schema.name,
            mlflow_uri= config2.tracking_uri,
           
        )

        return model_evaluation_config


IndentationError: expected an indented block after function definition on line 14 (2523006218.py, line 18)

In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as numpy
import joblib
import json

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_experiment("First Experiment")
        
        mlflow.set_tracking_uri(uri=self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            # with open(Path(self.config.metric_file_name), 'w') as outfile:
            #     json.dump(scores, outfile)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            # mlflow.sklearn.log_model(model, 'ElasticNet')


            # # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [18]:
try:
    config = ConfigurationManager()
    config.set_env_variables()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model.e
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e

[2024-12-27 14:41:49,173: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-27 14:41:49,180: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-27 14:41:49,192: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-27 14:41:49,196: INFO: common: created directory at: artifacts]
[2024-12-27 14:41:49,199: INFO: common: created directory at: artifacts/model_evaluation]


BoxKeyError: "'ConfigBox' object has no attribute 'tracking_uri'"